# Benchmarking on simulated binary classification data generated using sklearn's make_classification

## Overview
This notebook displays the percentage of informative features captured as well as noise output by each of the six feature selection methods tested (LLpowershap, powershap, BorutaShap, shapicant, Chi² squared and F test).

Adapted from the notebook by the authors of the Powershap python package available at https://github.com/predict-idlab/powershap

Orignal Notebook: https://github.com/predict-idlab/powershap/blob/main/examples/Benchmark_datasets.ipynb



## Dataset Information
- Simulated datasets of sample sizes 5000 and 10000


## Notes
- None


In [10]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import shap
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import sys
import time

from powershap import PowerShap
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


def benchmark_dict_print(output_dict):
    for key in output_dict:
        #samples
        print(key+": ")
        for key_inner in output_dict[key]:
            #features
            print(key_inner+": ")
            
            #per informative
            time_str = ""
            
            i = 0
            for avg_time in output_dict[key][key_inner]["Average time"]:
                
                i = i+1
                
                if i ==len(output_dict[key][key_inner]["Average time"]):
                    time_str = time_str + str(avg_time)+"s | "
                else:
                    time_str = time_str + str(avg_time)+"s - "
                  
            key_strs = ""
            for key_inner_inner in output_dict[key][key_inner]: 
                if key_inner_inner != 'Average time':
                    key_strs = key_strs + str(output_dict[key][key_inner][key_inner_inner]["found_informative_features"]) + " ("+str(output_dict[key][key_inner][key_inner_inner]["outputted_noise_features"])+") / "+str(output_dict[key][key_inner][key_inner_inner]["informative_features"]) + " | "
                
            print(time_str+key_strs)
                 

def output_dict_to_df(output_dict):
    df = pd.DataFrame(columns=["n_samples","total_features","informative_features","time","seed","found_informative_features","outputted_noise_features"])
    for samples in output_dict.keys():
        for features in output_dict[samples].keys():
            i = 0
            for informative in output_dict[samples][features].keys():
                if informative != 'Average time':
                    temp = pd.DataFrame.from_dict(output_dict[samples][features][informative])
                    temp["n_samples"]=int(samples)
                    temp["total_features"]=int(features)
                    temp = temp.reset_index()
                    temp = temp.rename(columns={"index":"seed"})
                    temp["time"]=output_dict[samples][features]['Average time'][i]
                    i = i+1
                    df = pd.concat([df,temp]) 

    return df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Benchmarking

In [12]:
sub = 'xgb_250_git' # folder

## LLpowershap

In [13]:
for hypercube in [True]:
    output_dict = {}
    for n_samples in [5000, 10000]:
        output_dict[str(n_samples)]={}
        for n_features in [20,100,250,500]:
            output_dict[str(n_samples)][str(n_features)]={}
            
            average_times = []
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            for n_informative in [3,10,33,50,90]:
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
                print("Amount of samples = "+str(n_samples))
                print("Total used features = "+str(n_features))
                n_inform = int(n_informative/100*n_features)
                if n_inform < 1:
                    n_inform = 1
                print("Informative features: "+str(n_inform)+" ("+str(n_informative)+"%)")
                print("")
                
                found_features = []
                found_idx_features = []
                times = []
                for random_seed in [0,1,2,3,4]:
                    print("Seed "+str(random_seed))
                    
                    if n_inform == 1:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=1, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed, n_clusters_per_class=1)
                    else:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=n_inform, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)                        
                    
                    X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
    
                    start_time = time.time()

                    selector = PowerShap(model=XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0),
                    automatic=True, method='LLpowershap')
                    selector.fit(X, y)
                    
                    times.append(time.time() - start_time)
                    
                    processed_shaps_df = selector._processed_shaps_df
                    print(50*"-")
                    
                    found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                    found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                    
                found_informative_features = [np.sum(np.isin(X.columns.values[:n_inform],f_list)) for f_list in found_idx_features]
                found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:n_inform])) for f_list in found_idx_features]
                print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
                print("Found features: "+str(found_features))
                print("Found "+str(np.mean(found_informative_features))+" of "+str(n_inform)+" informative features")
                print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
                
                average_times.append(np.round(times,2))
                
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=n_inform
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
                
                print(100*"=")
                
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            benchmark_dict_print(output_dict)
            print(100*"=")
        
    if not hypercube:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_LLpowershap_output_df.csv",index=False)
    else:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_LLpowershap_output_df_hypercube.csv",index=False)

Amount of samples = 5000
Total used features = 20
Informative features: 1 (3%)

Seed 0


  0%|          | 0/20 [00:00<?, ?it/s]

--------------------------------------------------
Seed 1


  0%|          | 0/20 [00:00<?, ?it/s]

--------------------------------------------------
Seed 2


...

  0%|          | 0/10 [00:00<?, ?it/s]

 97%|=================== | 1949/2000 [00:19<00:00]       

  0%|          | 0/10 [00:00<?, ?it/s]

 97%|=================== | 1949/2000 [00:19<00:00]       

--------------------------------------------------
Average time: 1499.01 seconds
Found features: [376, 352, 394, 368, 382]
Found 374.4 of 450 informative features
0.0 of 374.4 outputted powershap features are noise features
5000: 
20: 
[10.52  9.55  8.5  12.78  9.23]s - [11.38 13.05 12.34 10.93 13.63]s - [28.58 29.18 40.91 34.34 28.85]s - [59.07 55.66 41.43 48.52 70.42]s - [89.58 82.32 77.96 72.24 80.08]s | [1, 1, 1, 1, 1] ([0, 0, 0, 0, 0]) / 1 | [2, 2, 2, 2, 2] ([0, 0, 0, 0, 0]) / 2 | [6, 6, 6, 6, 6] ([0, 0, 0, 0, 0]) / 6 | [10, 10, 10, 10, 10] ([0, 0, 0, 0, 0]) / 10 | [18, 18, 18, 18, 18] ([0, 0, 0, 0, 0]) / 18 | 
100: 
[19.56 16.49 17.97 17.12 54.21]s - [59.15 50.65 44.92 42.14 64.94]s - [100.98  86.17  95.23  83.01  86.94]s - [107.73 112.12 219.48 100.38 110.9 ]s - [187.78 190.99 212.34 127.26 132.46]s | [3, 3, 3, 3, 3] ([0, 0, 0, 0, 0]) / 3 | [10, 10, 10, 10, 10] ([0, 0, 0, 0, 0]) / 10 | [33, 33, 33, 33, 33] ([0, 0, 0, 0, 0]) / 33 | [50, 50, 50, 50, 50] ([0, 0, 1, 0, 0]) / 50 | [8

## powershap

In [14]:
for hypercube in [True]:
    output_dict = {}
    for n_samples in [5000, 10000]:
        output_dict[str(n_samples)]={}
        for n_features in [20,100,250,500]:
            output_dict[str(n_samples)][str(n_features)]={}
            
            average_times = []
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            for n_informative in [3,10,33,50,90]:
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
                print("Amount of samples = "+str(n_samples))
                print("Total used features = "+str(n_features))
                n_inform = int(n_informative/100*n_features)
                if n_inform < 1:
                    n_inform = 1
                print("Informative features: "+str(n_inform)+" ("+str(n_informative)+"%)")
                print("")
                
                found_features = []
                found_idx_features = []
                times = []
                for random_seed in [0,1,2,3,4]:
                    
                    print("Seed "+str(random_seed))
                    
                    if n_inform == 1:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=1, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed, n_clusters_per_class=1)
                    else:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=n_inform, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)                        
                    
                    X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
    
                    start_time = time.time()

                    selector = PowerShap(model = XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0),
                                         automatic=True, method='powershap')
                    selector.fit(X, y)
                    
                    times.append(time.time() - start_time)
                    
                    processed_shaps_df = selector._processed_shaps_df
                    print(50*"-")
                    
                    found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                    found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                    
                found_informative_features = [np.sum(np.isin(X.columns.values[:n_inform],f_list)) for f_list in found_idx_features]
                found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:n_inform])) for f_list in found_idx_features]
                print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
                print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found "+str(np.mean(found_informative_features))+" of "+str(n_inform)+" informative features")
                print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
                
                average_times.append(np.round(times,2))
                
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=n_inform
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
                
                print(100*"=")
                
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            benchmark_dict_print(output_dict)
            print(100*"=")
        
    if not hypercube:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_powershap_output_df.csv",index=False)
    else:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_powershap_output_df_hypercube.csv",index=False)

Amount of samples = 5000
Total used features = 20
Informative features: 1 (3%)

Seed 0


  0%|          | 0/10 [00:00<?, ?it/s]

--------------------------------------------------
Seed 1


  0%|          | 0/10 [00:00<?, ?it/s]

--------------------------------------------------
Seed 2


...

--------------------------------------------------
Seed 4


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

--------------------------------------------------
Average time: 294.54 seconds
Found features: [392, 402, 392, 408, 374]
Found 393.6 of 450 informative features
0.0 of 393.6 outputted powershap features are noise features
5000: 
20: 
[2.5  2.26 4.88 2.23 2.54]s - [2.47 2.87 2.77 2.39 2.68]s - [3.65 3.4  4.32 3.46 3.84]s - [9.82 4.96 4.33 4.41 5.81]s - [7.22 6.33 6.15 6.06 6.68]s | [1, 1, 1, 1, 1] ([0, 0, 0, 0, 0]) / 1 | [2, 2, 2, 2, 2] ([1, 0, 1, 0, 0]) / 2 | [6, 6, 6, 6, 6] ([0, 0, 0, 0, 0]) / 6 | [10, 10, 10, 10, 10] ([1, 0, 0, 0, 0]) / 10 | [18, 18, 18, 18, 18] ([0, 0, 0, 0, 0]) / 18 | 
100: 
[9.93 4.65 5.89 4.83 9.23]s - [10.34 11.88  9.93  9.59 15.83]s - [14.18 24.82 12.89 13.7  26.82]s - [14.09 14.06 16.61 20.45 24.35]s - [16.76 16.11 30.02 16.49 16.15]s | [3, 3, 3, 3, 3] ([0, 0, 1, 1, 2]) / 3 | [10, 10, 10, 10, 10] ([5, 2, 2, 1, 6]) / 10 | [33, 33, 33, 33, 33] ([1, 1, 3, 3, 5]) / 33 | [50, 50, 50, 50, 50] ([2, 0, 4, 0, 5]) / 50 | [90, 89, 90, 90, 90] ([0, 1, 1, 0, 1]) / 90 | 
2

## BorutaShap

In [15]:
from BorutaShap import BorutaShap

for hypercube in [True]:
    output_dict = {}
    for n_samples in [5000, 10000]:
        output_dict[str(n_samples)]={}
        for n_features in [20,100,250,500]:
            output_dict[str(n_samples)][str(n_features)]={}
            
            average_times = []
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            for n_informative in [3,10,33,50,90]:
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
                print("Amount of samples = "+str(n_samples))
                print("Total used features = "+str(n_features))
                n_inform = int(n_informative/100*n_features)
                if n_inform < 1:
                    n_inform = 1
                print("Informative features: "+str(n_inform)+" ("+str(n_informative)+"%)")
                print("")
                
                found_features = []
                found_idx_features = []
                times = []
                for random_seed in [0,1,2,3,4]:
                    print("Seed "+str(random_seed))
                    
                    if n_inform == 1:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=1, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed, n_clusters_per_class=1)
                    else:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=n_inform, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)                        
                    
                    X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
    
                    start_time = time.time()

                    selector = BorutaShap(model =XGBClassifier(n_estimators=250, verbosity=0),
                                          importance_measure='shap',classification=True)
                    
                    selector.fit(X=X, y=y, verbose=False)
                    subset = selector.Subset()
                    
                    times.append(time.time() - start_time)
                    print(50*"-")
                    
                    found_features.append(len(subset.columns))
                    found_idx_features.append(subset.columns)
                    
                found_informative_features = [np.sum(np.isin(X.columns.values[:n_inform],f_list)) for f_list in found_idx_features]
                found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:n_inform])) for f_list in found_idx_features]
                print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
                print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found "+str(np.mean(found_informative_features))+" of "+str(n_inform)+" informative features")
                print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
                
                average_times.append(np.round(times,2))
                
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=n_inform
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
                
                print(100*"=")
        
    if not hypercube:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_borutashap_output_df.csv",index=False)
    else:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_borutashap_output_df_hypercube.csv",index=False)

Amount of samples = 5000
Total used features = 20
Informative features: 1 (3%)

Seed 0


  0%|          | 0/20 [00:00<?, ?it/s]

--------------------------------------------------
Seed 1


  0%|          | 0/20 [00:00<?, ?it/s]

--------------------------------------------------
Seed 2


...

--------------------------------------------------
Seed 3


  0%|          | 0/20 [00:00<?, ?it/s]

--------------------------------------------------
Seed 4


  0%|          | 0/20 [00:00<?, ?it/s]

--------------------------------------------------
Average time: 479.38 seconds
Found features: [117, 144, 156, 137, 123]
Found 135.4 of 450 informative features
0.0 of 135.4 outputted powershap features are noise features


## Shapicant

In [16]:
import shapicant

for hypercube in [True]:
    output_dict = {}
    for n_samples in [5000, 10000]:
        output_dict[str(n_samples)]={}
        for n_features in [20,100,250,500]:
            output_dict[str(n_samples)][str(n_features)]={}
            
            average_times = []
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            for n_informative in [3,10,33,50,90]:
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
                print("Amount of samples = "+str(n_samples))
                print("Total used features = "+str(n_features))
                n_inform = int(n_informative/100*n_features)
                if n_inform < 1:
                    n_inform = 1
                print("Informative features: "+str(n_inform)+" ("+str(n_informative)+"%)")
                print("")
                
                found_features = []
                found_idx_features = []
                times = []
                for random_seed in [0,1,2,3,4]:
                    print("Seed "+str(random_seed))
                    
                    if n_inform == 1:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=1, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed, n_clusters_per_class=1)
                    else:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=n_inform, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)                        
                    
                    X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
                    
                    X["class"]=y
                    X = X.reset_index()

                    explainer_type = shap.TreeExplainer
                    # if classification is False it is a Regression problem
                    model = XGBClassifier(n_estimators=250, verbosity=0)
                    selector = shapicant.PandasSelector(model, explainer_type, random_state=42)
    
                    train_idx,val_idx = train_test_split(X["index"].values,test_size=0.2,random_state = 0)
    
                    X_train = X[X["index"].isin(train_idx)].copy(deep=True)[list(X.columns.values[1:-1])]
                    X_val = X[X["index"].isin(val_idx)].copy(deep=True)[list(X.columns.values[1:-1])]
                    Y_train =  X[X["index"].isin(train_idx)]["class"]                    
    
                    start_time = time.time()
                    
                    selector.fit(X_train, Y_train, X_validation=X_val)
    
                    subset = selector.get_features()
                    p_values = selector.p_values_
    
                    np.array(subset)
    
                    times.append(time.time() - start_time)
                    print(50*"-")
                    
                    found_features.append(len(subset))
                    found_idx_features.append(subset)

                    X.drop(columns=['index'], inplace=True)
                    
                found_informative_features = [np.sum(np.isin(X.columns.values[:n_inform],f_list)) for f_list in found_idx_features]
                found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:n_inform])) for f_list in found_idx_features]
                print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
                print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found "+str(np.mean(found_informative_features))+" of "+str(n_inform)+" informative features")
                print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
                
                average_times.append(np.round(times,2))
                
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=n_inform
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
                
                print(100*"=")
                
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            benchmark_dict_print(output_dict)
            print(100*"=")
        
    if not hypercube:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_shapicant_output_df.csv", index=False)
    else:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_shapicant_output_df_hypercube.csv", index=False)

Amount of samples = 5000
Total used features = 20
Informative features: 1 (3%)

Seed 0


Computing null SHAP values: 100%|████████████████████████████████████████████████████| 100/100 [01:32<00:00,  1.08it/s]


--------------------------------------------------
Seed 1


Computing null SHAP values: 100%|████████████████████████████████████████████████████| 100/100 [01:31<00:00,  1.09it/s]


--------------------------------------------------
Seed 2


...

--------------------------------------------------
Seed 3


Computing null SHAP values: 100%|████████████████████████████████████████████████████| 100/100 [29:47<00:00, 17.87s/it]


--------------------------------------------------
Seed 4


Computing null SHAP values: 100%|████████████████████████████████████████████████████| 100/100 [29:50<00:00, 17.90s/it]

--------------------------------------------------
Average time: 1790.81 seconds
Found features: [154, 158, 143, 144, 144]
Found 148.6 of 450 informative features
0.0 of 148.6 outputted powershap features are noise features
5000: 
20: 
[92.57 91.78 92.13 91.79 92.38]s - [91.58 92.72 91.71 93.15 92.79]s - [92.11 92.2  92.08 92.53 92.16]s - [92.33 92.18 93.04 92.56 92.48]s - [92.6  92.6  92.69 92.07 92.64]s | [1, 1, 1, 1, 1] ([3, 6, 4, 4, 0]) / 1 | [2, 2, 2, 2, 2] ([3, 1, 2, 2, 2]) / 2 | [6, 6, 6, 6, 6] ([0, 0, 1, 0, 0]) / 6 | [10, 10, 10, 10, 10] ([0, 0, 0, 0, 0]) / 10 | [18, 18, 17, 17, 18] ([0, 0, 0, 0, 0]) / 18 | 
100: 
[222.23 226.16 221.94 224.44 223.1 ]s - [223.56 222.74 224.17 223.16 223.4 ]s - [222.89 223.48 222.49 223.72 222.66]s - [222.85 222.94 224.15 222.78 222.79]s - [223.8  223.14 223.37 222.97 222.7 ]s | [3, 3, 3, 3, 3] ([3, 4, 5, 4, 3]) / 3 | [10, 10, 10, 10, 10] ([0, 0, 0, 0, 0]) / 10 | [30, 31, 29, 28, 30] ([0, 0, 0, 0, 0]) / 33 | [35, 37, 33, 37, 32] ([0, 0, 1, 0, 0])

## Chi-squared

In [17]:
from sklearn.feature_selection import chi2

for hypercube in [True]:
    output_dict = {}
    for n_samples in [5000, 10000]:
        output_dict[str(n_samples)]={}
        for n_features in [20,100,250,500]:
            output_dict[str(n_samples)][str(n_features)]={}
            
            average_times = []
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            for n_informative in [3,10,33,50,90]:
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
                print("Amount of samples = "+str(n_samples))
                print("Total used features = "+str(n_features))
                n_inform = int(n_informative/100*n_features)
                if n_inform < 1:
                    n_inform = 1
                print("Informative features: "+str(n_inform)+" ("+str(n_informative)+"%)")
                print("")
                
                found_features = []
                found_idx_features = []
                times = []
                for random_seed in [0,1,2,3,4]:
                    print("Seed "+str(random_seed))
                    
                    if n_inform == 1:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=1, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed, n_clusters_per_class=1)
                    else:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=n_inform, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)                        
                    
                    X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
    
                    start_time = time.time()

                    X = X+np.abs(X.min())
                    selected_features = list(X.columns.values[np.where(chi2(X,y)[1]<0.01)[0]])
                    
                    times.append(time.time() - start_time)

                    print(50*"-")
                    
                    found_features.append(len(selected_features))
                    found_idx_features.append(selected_features)
                    
                found_informative_features = [np.sum(np.isin(X.columns.values[:n_inform],f_list)) for f_list in found_idx_features]
                found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:n_inform])) for f_list in found_idx_features]
                print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
                print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found "+str(np.mean(found_informative_features))+" of "+str(n_inform)+" informative features")
                print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
                
                average_times.append(np.round(times,2))
                
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=n_inform
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
                
                print(100*"=")
                
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            benchmark_dict_print(output_dict)
            print(100*"=")
        
    if not hypercube:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_chi_squared_output_df.csv",index=False)
    else:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_chi_squared_output_df_hypercube.csv",index=False)

Amount of samples = 5000
Total used features = 20
Informative features: 1 (3%)

Seed 0
--------------------------------------------------
Seed 1
--------------------------------------------------
Seed 2
--------------------------------------------------
Seed 3
--------------------------------------------------
Seed 4
--------------------------------------------------
Average time: 0.0 seconds
Found features: [1, 1, 1, 1, 1]
Found 1.0 of 1 informative features
0.0 of 1.0 outputted powershap features are noise features
Amount of samples = 5000
Total used features = 20
Informative features: 2 (10%)

Seed 0
--------------------------------------------------
Seed 1
--------------------------------------------------
Seed 2
--------------------------------------------------
Seed 3
--------------------------------------------------
Seed 4
--------------------------------------------------
Average time: 0.0 seconds
Found features: [1, 1, 1, 1, 1]
Found 1.0 of 2 informative features
0.0 of 1.0 o

# F Test

In [18]:
from sklearn.feature_selection import f_classif

for hypercube in [True]:
    output_dict = {}
    for n_samples in [5000, 10000]:
        output_dict[str(n_samples)]={}
        for n_features in [20,100,250,500]:
            output_dict[str(n_samples)][str(n_features)]={}
            
            average_times = []
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            for n_informative in [3,10,33,50,90]:
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
                print("Amount of samples = "+str(n_samples))
                print("Total used features = "+str(n_features))
                n_inform = int(n_informative/100*n_features)
                if n_inform < 1:
                    n_inform = 1
                print("Informative features: "+str(n_inform)+" ("+str(n_informative)+"%)")
                print("")
                
                found_features = []
                found_idx_features = []
                times = []
                for random_seed in [0,1,2,3,4]:
                    print("Seed "+str(random_seed))
                    
                    if n_inform == 1:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=1, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed, n_clusters_per_class=1)
                    else:
                        X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=n_inform, n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)                        
                    
                    X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
                    
                    start_time = time.time()

                    selected_features = list(X.columns.values[np.where(f_classif(X,y)[1]<0.01)[0]])
                    
                    times.append(time.time() - start_time)
                    
                    print(50*"-")
                    
                    found_features.append(len(selected_features))
                    found_idx_features.append(selected_features)
                    
                found_informative_features = [np.sum(np.isin(X.columns.values[:n_inform],f_list)) for f_list in found_idx_features]
                found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:n_inform])) for f_list in found_idx_features]
                print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
                print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found "+str(np.mean(found_informative_features))+" of "+str(n_inform)+" informative features")
                print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
                
                average_times.append(np.round(times,2))
                
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=n_inform
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
                
                print(100*"=")
                
            output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
            benchmark_dict_print(output_dict)
            print(100*"=")
        
    if not hypercube:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_f_test_output_df.csv",index=False)
    else:
        output_dict_to_df(output_dict).to_csv(f"../results/{sub}/make_classi_f_test_output_df_hypercube.csv",index=False)

Amount of samples = 5000
Total used features = 20
Informative features: 1 (3%)

Seed 0
--------------------------------------------------
Seed 1
--------------------------------------------------
Seed 2
--------------------------------------------------
Seed 3
--------------------------------------------------
Seed 4
--------------------------------------------------
Average time: 0.0 seconds
Found features: [1, 2, 1, 1, 3]
Found 1.0 of 1 informative features
0.6 of 1.6 outputted powershap features are noise features
Amount of samples = 5000
Total used features = 20
Informative features: 2 (10%)

Seed 0
--------------------------------------------------
Seed 1
--------------------------------------------------
Seed 2
--------------------------------------------------
Seed 3
--------------------------------------------------
Seed 4
--------------------------------------------------
Average time: 0.0 seconds
Found features: [1, 2, 1, 1, 1]
Found 1.0 of 2 informative features
0.2 of 1.2 o